## LSTM

In [16]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [17]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [18]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


#### validation set

In [19]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


#### Utility functions to map characters to vocabulary IDs and back.

In [20]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) # string => ASCII / fist letter => reorder to ndex of ASCII code '0'

# if input is char type           => ASCII code index
# if          blank or unexpected => 0
def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

# if input is dictionary key           => corresponding ASCII character
# if          not dictionary keyblank or unexpectec => blank
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1) #  reorder to index of ASCII code '0'
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


#### Function to generate a training batch for LSTM model

In [21]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        
        segment = self._text_size // batch_size # number of batch
        self._cursor = [ offset * segment for offset in range(batch_size)]
        # [0 * seg, 1 * seg, 2 * seg, ... ,  (batch_size - 1) * seg]
        
        self._last_batch = self._next_batch()
        
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data"""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype = np.float)
        # shape = batch_size x vocabulary size
        
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b] # current position of cursor
                                       ])] = 1.0        # position where new cursor starts => 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    
    def next(self):
        """Generate the next array of batches from the data.
        The array consists of the last batch of the previous array, folllowed by num_unrollings new ones."""
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible characters
    back into its (most likely) character representation"""
    return [id2char(c) for c in np.argmax(probabilities, 1)]
    
def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string representation"""
    s = [''] * batches[0].shape[0] # '' 으로 가득찬 batch-size 크기의 리스트
    for b in batches:
        s = [''.join(x) # 모든 characters 의 결과들(문자)을 하나로 연결하여 출력 => string
             for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 2, 5)

print(batches2string(train_batches.next()), '\n')
print(batches2string(train_batches.next()), '\n')
print(batches2string(valid_batches.next()), '\n')
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad'] 

['ists advoca', 'ary governm', 'hes n